In [1]:
#default_exp segmentation.metrics

In [2]:
%autosave 60 
import fastai; print(fastai.__version__)

Autosaving every 60 seconds
1.0.58.dev0


In [3]:
#export
from fastai.vision import *
from fastai.metrics import foreground_acc, dice

In [4]:
#export
_all_ = ["foreground_acc", "dice"]

### test

In [5]:
from local.test import *

In [6]:
a = tensor([[
    [
    [0,0,1],
    [0,0,1],
    [1,1,1]
],
    [
    [1,1,0],
    [1,1,0],
    [0,0,0]
]
]])

b = tensor([[[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]]])

a.size(), b.size()

(torch.Size([1, 2, 3, 3]), torch.Size([1, 1, 3, 3]))

In [7]:
test_eq(dice(a,b, eps=1e-8), (2*1)/(8))

### `iou`

In [8]:
#export
def iou(input: torch.Tensor, targs: torch.Tensor, **kwargs)->Rank0Tensor:
    "Binary IOU"
    return dice(input, targs, iou=True, **kwargs)

In [9]:
test_eq(iou(a,b, eps=1e-8), 1/7)

### `multilabel_dice`

In [19]:
x = torch.randn((1,5,12,12))
x = x.sigmoid()

In [24]:
x

tensor([[[[0.7891, 0.4816, 0.8353, 0.3786, 0.7060, 0.7096, 0.1475, 0.5914,
           0.9412, 0.6177, 0.1099, 0.4880],
          [0.2022, 0.8361, 0.2771, 0.4903, 0.4966, 0.4639, 0.7465, 0.5042,
           0.4963, 0.5274, 0.6934, 0.5535],
          [0.7535, 0.1464, 0.1533, 0.2185, 0.6906, 0.4787, 0.3827, 0.4130,
           0.5057, 0.2122, 0.3575, 0.5777],
          [0.6759, 0.7170, 0.7563, 0.5696, 0.5087, 0.3745, 0.2189, 0.7463,
           0.2984, 0.7862, 0.4417, 0.5718],
          [0.5699, 0.3516, 0.3363, 0.3930, 0.7089, 0.4816, 0.1969, 0.8179,
           0.2691, 0.5027, 0.5887, 0.6030],
          [0.5320, 0.2512, 0.8861, 0.2164, 0.8254, 0.7881, 0.2061, 0.2746,
           0.6296, 0.4460, 0.2627, 0.7392],
          [0.6771, 0.3638, 0.7475, 0.8795, 0.3283, 0.8059, 0.3292, 0.8297,
           0.3280, 0.7454, 0.5338, 0.4945],
          [0.2815, 0.7597, 0.5456, 0.2196, 0.4745, 0.1634, 0.3610, 0.6653,
           0.8685, 0.7357, 0.4956, 0.3986],
          [0.3234, 0.1543, 0.6976, 0.6465, 0.334

In [23]:
(x).argmax(1)

tensor([[[3, 1, 1, 1, 3, 0, 1, 2, 0, 3, 2, 4],
         [3, 0, 1, 3, 4, 0, 2, 2, 2, 4, 0, 3],
         [0, 2, 3, 1, 3, 3, 3, 1, 3, 1, 3, 1],
         [4, 0, 4, 4, 2, 4, 1, 0, 1, 0, 1, 0],
         [3, 3, 3, 4, 3, 1, 2, 0, 2, 1, 1, 0],
         [3, 4, 1, 1, 0, 0, 4, 4, 3, 4, 2, 0],
         [2, 3, 3, 0, 4, 0, 4, 0, 1, 4, 1, 2],
         [2, 0, 3, 3, 1, 1, 2, 1, 0, 0, 1, 2],
         [4, 1, 0, 3, 2, 0, 4, 0, 0, 2, 0, 3],
         [1, 1, 1, 4, 1, 1, 2, 0, 4, 3, 0, 0],
         [3, 0, 2, 2, 3, 1, 0, 0, 4, 0, 1, 3],
         [0, 3, 2, 3, 0, 0, 3, 2, 2, 0, 1, 1]]])

In [9]:
#export
def multilabel_dice(input:Tensor, targs:Tensor, c:int, iou:bool=False, 
                    mean=True, eps:float=1e-8, sigmoid:bool=False, threshold:float=0.5)->Rank0Tensor:
    "Batch/Dataset Mean Dice"
    if sigmoid:
        input.sigmoid() > threshold
    else:
        input = input.argmax(dim=1, keepdim=True).view(-1)
        
    targs = targs.view(-1)
    res = []
    for ci in range(c):
        _input, _targs = input == ci, targs == ci
        intersect = (_input * _targs).sum().float()
        union = (_input+_targs).sum().float()
        if not iou: res.append((2. * intersect / union if union > 0 else union.new([1.]).squeeze()))
        else: res.append(intersect / (union-intersect+eps))
    res = torch.tensor(res).to(input.device)
    if not mean: return res
    else: return res.mean()

In [10]:
test_eq(multilabel_dice(a,b,2, eps=1e-8), ((2*1)/(8) + (2*2)/(10))/2)

### `multilabel_iou`

In [11]:
#export
def multilabel_iou(input: torch.Tensor, targs: torch.Tensor, c)->Rank0Tensor:
    "Batch/Dataset Mean IOU"
    return multilabel_dice(input, targs, c=c, iou=True)

In [12]:
test_eq(multilabel_iou(a,b,2), (1/7+2/8)/2)

### export

In [5]:
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_script.ipynb.
Converted 02_scheduler.ipynb.
Converted 03_callbacks.ipynb.
Converted 10_segmentation_dataset.ipynb.
Converted 11_segmentation_losses_mulitlabel.ipynb.
Converted 11b_segmentation_losses_binary.ipynb.
Converted 12_segmentation_metrics.ipynb.
Converted 13_segmentation_models.ipynb.
Converted segmentation_training.ipynb.


### fin